In [2]:
%env CUDA_VISIBLE_DEVICES=2
import sys
sys.path.append("/home/isprovilkov/rupo")
import rupo.api
import tensorflow as tf
from nltk.translate.bleu_score import corpus_bleu

sess = tf.InteractiveSession()

env: CUDA_VISIBLE_DEVICES=2


Using TensorFlow backend.


In [2]:
MODEL_ROOT = '' # If your data is placed near repository folder.
MODEL_PATH = MODEL_ROOT + 'translation-model.pkl' # Your model path here
MODEL_PATH = 


RUPO_DATA_ROOT = '../../data/rupo/' # '/srv/hd6/data/Poem2Poem/data/rupo/'
RUPO_STRESS_MODEL_PATH   = RUPO_DATA_ROOT + \
                           'stress_models/stress_ru_LSTM64_dropout0.2_acc99_wer8.h5'
RUPO_ZALYZNIAK_DICT_PATH = RUPO_DATA_ROOT + 'dict/zaliznyak.txt'
del RUPO_DATA_ROOT

MODEL_BPE_PATH = MODEL_ROOT + 'translation-model-bpe-rules.txt'

In [3]:
from subword_nmt.apply_bpe import BPE

bpe_en = BPE(open(MODEL_BPE_PATH, mode = 'r'))

ENGLISH_CHARS = set([chr(n) for n in range(ord('A'), ord('Z') + 1)] + \
                    [chr(n) for n in range(ord('a'), ord('z') + 1)] + [' ', '@', '-', "'"])

def get_reversed(line):
    return ''.join(reversed(line))

def filter_out_chars(line, possible_chars):
    return u''.join([c for c in line if c in possible_chars])

def process_en_poem(poem):
    lines = poem.strip().lower().split('\n')
    lines = [filter_out_chars(l, ENGLISH_CHARS) for l in lines]
    lines = [get_reversed(l) for l in lines]
    lines = [bpe_en.process_line(line.strip()) for line in lines]
    return [get_reversed(l) for l in lines]

In [4]:
import os
import sys
sys.path.append(os.path.abspath('../models'))

from rhyme_translator import RhymeType, RhymeTranslator
from translation_model import AttentiveModel

In [5]:
def make_rupo_engine():
    rupo_engine = rupo.api.Engine(language = 'ru')
    rupo_engine.load(stress_model_path = RUPO_STRESS_MODEL_PATH,
                     zalyzniak_dict = RUPO_ZALYZNIAK_DICT_PATH)
    return rupo_engine

global_rupo_engine = make_rupo_engine()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
if 'model_loaded' in globals():
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    # Need to also recreate Rupo as it uses TensorFlow:
    global_rupo_engine = make_rupo_engine()

model_loaded = AttentiveModel(sess, MODEL_PATH)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [11]:
translator = RhymeTranslator(model_loaded, global_rupo_engine)

In [12]:
assert global_rupo_engine.is_rhyme('вещи', 'ветер') # This is how Rupo thinks

lines = process_en_poem('''Your voice
Called me outside the window
Of uncontested summer all things raise
seamless air''')
print('\n'.join(translator.translate_lines(lines)))
print()
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      rhyme_test_counts=(5, 5, 2),
                                                      max_total_rhyme_tests = 1000)))

Английский голос.
прямо у меня за окном.
Несмотря на то лето, как засчитывает все вещи,
Горячий воздух.

Английский голос.
прямо у меня за окном.
Несмотря на то лето, как засчитывает все вещи,
ветер.


In [13]:
lines = process_en_poem('''Torches are made to light, jewels to wear,
Dainties to taste, fresh beauty for the use,
Herbs for their smell, and sappy plants to bear;
Things growing to themselves are growth’s abuse,
Seeds spring from seeds, and beauty breedeth beauty;
Thou wast begot; to get it is thy duty.''')
print('\n'.join(translator.translate_lines(lines)))
print()
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      rhyme_test_counts=(5, 5, 2),
                                                      max_total_rhyme_tests = 1000)))
print()
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      rhyme_test_counts=(5, 5, 2),
                                                      max_total_rhyme_tests = 1000)))

Мы должны сделать свои драгоценные драгоценности в "Прости".
- для свежей красоты для "Пустяки",
Несмотря на их, и для птиц. Девочки из "я".
Несмотря на себя, пожирающий для себя для себя - "я".
Весна из Герцогина и королевская пехота: "Ну-ка".
Ты, бл*, для меня, - это твоя "волна!".

Мы должны сделать свои драгоценные драгоценности в "Прости".
Чтобы раствориться об простыней для красоты для непристойности,
Несмотря на их, и для птиц. Девочки из "я".
Несмотря на себя, пожирающий для себя для себя - "я".
Железные поляны из ресницы и красоты, урана.
Ты, бл*, для меня, - это твоя "волна!".

Я создана для того, чтобы попасть в райские драгоценности, чтобы разобраться с евреями.
- для свежей красоты для известности...
Они за их восхищённые и Милые цветы из плотины.
- очередные плотины...
Весна из Герцогина и королевская пехота: "Ну-ка".
Ты, бл*, для меня, - это твоя "волна!".


In [14]:
# Model internal translation function:
lines = list(map(get_reversed, process_en_poem('''
                  Let this wind blow through the night
                  And we are going away
                  The animal hasn't crossed the street
                  Because it was too tired''')))
translated = model_loaded.translate_lines(lines)[0]
print('\n'.join([get_reversed(line) for line in translated]).replace(' @@', ''))

Он говорит о том, что этот ветер прорывается сквозь ночь,
Ладно, мы уходим прочь.
Крупное животное проходил через улицу,
Нет, они так устали.


In [15]:
lines = process_en_poem('''Load up on guns and bring your friends
It's fun to lose and to pretend
She's over bored and self assured
Oh no, I know a dirty word''')
print('----ARGMAX----')
print('\n'.join(translator.translate_lines(lines)))
print()
print('----RHYME_WORD/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      rhyme_test_counts=(3, 10, 3),
                                                      max_total_rhyme_tests = 0)))
print()
print('----RHYME_SUFFIX/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      rhyme_test_counts=(3, 10, 3),
                                                      max_total_rhyme_tests = 0)))

----ARGMAX----
Приведи своих друзей
Это не так, весело, чтобы проиграть и притворяться.
Мне стало скучно и быть уверенным в себе.
"Ну-ка", я знаю непристойное слово,

----RHYME_WORD/ARGMAX----
Приведи своих друзей
Это не так, весело, чтобы проиграть и притворяться.
Мне стало скучно и был уверен.
"Человек", я знаю. Грязное словечко.

----RHYME_SUFFIX/ARGMAX----
Приведи своих друзей
Это не так, весело, чтобы проиграть и притворяться.
Мне стало скучно и быть уверенным в себе.
"Ну-ка", я знаю непристойное слово,


In [16]:
lines_ru = '''Крошка сын к отцу пришел,
и спросила кроха:
— Что такое хорошо
и что такое плохо?'''

# Translated to English with YandexTranslate
# And punctuation manually removed

lines = process_en_poem('''Baby son to his father came
and asked crumbs
What is good
what's wrong''')
print('----ARGMAX----')
print('\n'.join(translator.translate_lines(lines)))
print()
print('----RHYME_WORD/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 2500)))
print()
print('----RHYME_SUFFIX/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 2500)))

----ARGMAX----
Ваш сын вернулся к отцу.
Не спрашивай меня, и все крошки.
- это хорошо.
Это неправильно.

----RHYME_WORD/ARGMAX----
Он в том, что с сыном пришёл его отец.
и спросила у детей.
- это плохо.
Это не плохо.

----RHYME_SUFFIX/ARGMAX----
Он вернулся с сыном с отцом.
Попросила меня с ребёнком.
- прекрасно.
Это неправильно.


In [17]:
lines_ru = '''У меня секретов нет,
слушайте, детишки,
папы этого ответ
помещаю в книжке.'''

# Translated to English with YandexTranslate
# And punctuation manually removed

lines = process_en_poem('''I have no secrets
listen kids
dads this answer
I put it in the book''')

print('----ARGMAX----')
print('\n'.join(translator.translate_lines(lines)))
print()
print('----RHYME_WORD/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      # 5 variants of token at last position
                                                      # 5 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 5, 3, 5),
                                                      max_total_rhyme_tests = 0)))
print()
print('----RHYME_SUFFIX/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      # 5 variants of token at last position
                                                      # 5 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 5, 3, 5),
                                                      max_total_rhyme_tests = 0)))

----ARGMAX----
У меня нет секретов,
Ох, слушать, детей,
Приказ от отцов на этот ответ.
Положила её в книге,

----RHYME_WORD/ARGMAX----
У меня нет секретов,
Ох, слушать, детей,
Ответ от этих отцов, вот это.
Я внесла всю книгу в этом,

----RHYME_SUFFIX/ARGMAX----
У меня нет секретов,
Ох, слушать, детей,
Редента с этим ответом.
Я внесла всю книгу в этом,


In [18]:
lines_ru = '''У меня секретов нет,
слушайте, детишки,
папы этого ответ
помещаю в книжке.'''

# Translated to English with YandexTranslate
# And punctuation manually removed

lines = process_en_poem('''I have no secrets
listen kids
dads this answer
I put it in the book''')

print('----ARGMAX----')
print('\n'.join(translator.translate_lines(lines)))
print()
print('----RHYME_WORD/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 0)))
print()
print('----RHYME_SUFFIX/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 0)))

----ARGMAX----
У меня нет секретов,
Ох, слушать, детей,
Приказ от отцов на этот ответ.
Положила её в книге,

----RHYME_WORD/ARGMAX----
У меня нет друзей,
Ох, слушать, детей,
Они смотрят на это.
Я внесла всю книгу в этом,

----RHYME_SUFFIX/ARGMAX----
У меня нет секретов,
Послушай, мальчиков,
Редента с этим ответом.
Я внесла всю книгу в этом,


In [19]:
lines_ru = '''Этот в грязь полез и рад.
что грязна рубаха.
Про такого говорят:
он плохой, неряха.'''

# Translated to English with YandexTranslate
# And punctuation manually removed

lines = process_en_poem('''This in the dirt and got excited
that dirty shirt
About this say
he's bad sloppy''')

print('----ARGMAX----')
print('\n'.join(translator.translate_lines(lines)))
print()
print('----RHYME_WORD/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.WORD,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 0)))
print()
print('----RHYME_SUFFIX/ARGMAX----')
print('\n'.join(translator.translate_lines_with_rhyme(lines,
                                                      rhyme_type = RhymeType.SUFFIX,
                                                      # 1 variants of token at last position
                                                      # 20 variants at 'last minus 1' position
                                                      # 2 variants at 'last minus 2' position
                                                      rhyme_test_counts=(1, 20, 2),
                                                      max_total_rhyme_tests = 0)))

----ARGMAX----
Он врывается в грязь и взволнован.
Это эта коварная рубашка,
так говорят:
Ладно, это плохо.

----RHYME_WORD/ARGMAX----
Он врывается в грязь и взволнован.
Это такая грязная футболка,
так говорят:
Ладно, это плохо.

----RHYME_SUFFIX/ARGMAX----
Наверное, в грязи, и я была взволнована.
Это кожаная рубашкана,
Это то, что здесь сказано:
Очевидно.
